# Random Number Generators

For any Monte Carlo (MC) computational technique, random numbers (or pseudo-random numbers) are critical, not only in the sampling characterstics of the generator but also the speed of the sampling. For example, simulating a Large Hadron Collider (LHC) collission can require upwards of $2.5 \times 10^6$ random number calls.

This worksheet is largely based on chapter 7 of [Numerical Recipes](https://numerical.recipes/book.html), and associated primary papers. The following is a list of references for further reading.

*  Hammersley, J.M. and Handscomb, D.C. 1964, Monte Carlo Methods (London: Methuen)
* Kalos, M.H. and Whitlock, P.A. 1986, Monte Carlo Methods (New York: Wiley)
* Bratley, P., Fox, B.L., and Schrage, E.L. 1983, A Guide to Simulation, 2nd ed. (New York: Springer)
* Lepage, G.P. 1978, A New Algorithm for Adaptive Multidimensional Integration, Journal of Computational Physics, vol. 27, pp. 192-203
* Lepage, G.P. 1980, VEGAS: An Adaptive Multidimensional Integration Program, Publication CLNS-80/447, Cornell University
* Numerical Recipes Software 2007, [Complete VEGAS Code Listing](http://numerical.recipes/webnotes?9), Numerical Recipes Webnote No. 9
* Press, W.H., and Farrar, G.R. 1990, Recursive Stratified Sampling for Multidimensional Monte Carlo Integration, Computers in Physics, vol. 4, pp. 190-195
* Numerical Recipes Software 2007, [Complete Miser Code Listing](http://numerical.recipes/webnotes?10), Numerical Recipes Webnote No. 10

## Introduction

This tutorial will give an introduction to Random Number Generators (or RNGs, for short). For this tutorial it is useful to have interactive plotting via `matplotlib`. To enable this, we need to install the `ipympl` module and restart the kernel.

In [ ]:
# Install `ipympl`.
!pip install -q ipympl
get_ipython().kernel.do_shutdown(restart=True)

Next, we configure `matplotlib` and allow widgets in Colab.

In [ ]:
# Allow for interactive plots.
%matplotlib widget
# Allow for widgets in Colab.
try:
    from google.colab import output

    output.enable_custom_widget_manager()
except:
    pass

Now we are ready to go!

RNG algorithms are really pseudo-random (pRNG). This means they are deterministic. Randomness (Jaynes, [Probability Theory: The Logic of Science](http://www.med.mcgill.ca/epidemiology/hanley/bios601/GaussianModel/JaynesProbabilityTheory.pdf)) is a statement about knowledge, not about causes. pRNGs have causes, but you might not know them, or you might use them in a way so that they appear to be random, or random "enough".

What are the basic properties of RNs? On any interval, there is an equal probability to obtain any value. For convenience, we will take this interval to be $(0,1)$. Note, we will generally exclude $0$ from the interval when considering RNs on a computer, since many algorithms will fail if $0$ is ever encountered.

At the very least, we expect our computer RNs to have the properties of abstract ones.

Mean:

$$\langle x \rangle = \int_0^1 \text{d}x\, x = \frac{1}{2}$$

Variance:

$$\langle x^2 \rangle - \langle x \rangle^2 = \frac{1}{3}-\frac{1}{4} = \frac{1}{12}$$

Since it is useful to check the mean and variance, let us write simple functions which calculate these values given an iterable object like a list or array.

In [ ]:
# Note, these functions are available in `numpy` as `numpy.mean` and
# `numpy.var`.
def mean(vals):
    """
    Return the mean for a list of numbers given by `vals`.
    """
    return sum(vals) / float(len(vals))


def variance(vals):
    """
    Return the variance for a list of numbers given by `vals`.
    """
    return mean([val**2 for val in vals]) - mean(vals) ** 2

In practice, computers represent only a finite range of numbers.
This has implications for RNGs. Any algorithm that is deterministic and shuffles through the numbers is bound to repeat itself. Further, if our algorithm uses one number as the seed for the next, this repetition will mean that the sequence of pRNs repeats itself. This hardly looks random. Much of the art in developing good RNGs is knowing how to characterize or calculate the length of these cycles. If the cycle is long enough, much longer than any sequence of RNs we ever use in practice, then there is hope that this sequence will have the desired properties of random numbers. Identifying an algorithm with a long cycle is a priority, and possibly led to the focus on modular arithmetic, to be discussed shortly.

Before we get started, we need to make sure we have the `matplotlib` package configured as we need it. Since we will be looking at some 3D plots, we need to install the `ipympl` package and then set how `matplotlib` displays figures so that we can interact with them.

## Middle square method

The need for such algorithms only came about with the development of practical computers and Monte Carlo (MC) techniques. Historically, we are placing ourselves in the 1940-1950s.
Von Neumann (who else?) used a method called the [middle square](https://en.wikipedia.org/wiki/Middle-square_method). It can give us some insight to the problem we are facing.

The middle square algorithm is short and strange:

1. input a number of length $n$
2. square the number and zero-pad if not of length $2n$
3. output the "middle" number of length $n$
4. use this number as the random variate and the input for step 1

For example:

$$14^2 = 196 \to 0196 \to 19; 19^2 = 0361 \to 36; 36^2 = 1296 \to 29; 29^2 = 0841 \to 84, \text{ etc.}$$

Von Neumann used larger $n$ than this, but you get the point.

Before we practically explore the middle square method, let us first consider some technical realities behind RNGs. Perhaps the most critical aspect is that all RNGs require state; for the next RN to be generated, the RNG must have saved information stored on how the previous RN was generated. In the context of Python, this means that we need to use a class to define an RNG.

In [ ]:
# Here, we define a simple base class which can hold the state for an RNG.
class RNG:
    """
    This is a simple base class for random number generation.
    """

    def __init__(self, seed):
        """
        Constructor for this class, e.g. `RNG(seed)`.
        """
        self.seed = seed
        # In derived classes, further state can be initialized here.

    def __call__(self):
        """
        This defines the `()` for this class and should return an RN between 0 and
        1 for this generator. For example:
        ```
        rng = RNG(seed)
        rng()
        ```
        """
        return 0.0
        # In derived classes, the actual algorithm should be implemented here.

We are also interested in the periodicity for the RNG, so we can define a function, which given an RNG, generates RNs until the sequence repeats.

In [ ]:
# Since it is useful to determine the period of an RNG, let us define a method
# that does just this. Since we are storing the RNs we need to be careful not
# to store a very long list.
def rng_sequence(rng, n=int(1e6)):
    """
    Given an RNG, return its sequence after its first repition or until
    `n` RNs have been generated.
    """
    # Start with no RN and create the RN sequence.
    rn, rns = None, []

    # Loop while the next RN is not in the sequence.
    while rn not in rns:
        # Append the RN to the sequence.
        if rn != None:
            rns.append(rn)
        # Generate the next RN.
        rn = rng()
        # Return if `n` is reached.
        if len(rns) >= n:
            return rns

    # Return the sequence.
    return rns

### Exercise: implement the middle square method

For $n$=2, try all seeds (inputs) and identify the longest sequence (before a repeat). For this longest sequence, calculate the mean and variance. Then, identify the worst seeds (0 period).

In [ ]:
###START_EXERCISE
# All RNGs require state of some sort, i.e. they need to save information about
# the previous random number call. This means that in Python, we should write
# our RNGs as classes.


# Here, we define the middle square generator as a derived class of `RNG`.
class RNGMiddleSquare(RNG):
    # For this example `n` would be 2.
    def __init__(self, seed, n):
        # This initializes the base class.
        super().__init__(seed)
        # Additional state initialized here.
        self.n = n

    def __call__(self):
        # Return the actual number.
        return 0


# Get the seed and create the RNG.
seed = int(input("Please enter a two-digit number:\n[##] "))
rng = RNGMiddleSquare(seed, 2)

# Determine the period.
###STOP_EXERCISE

In [ ]:
###START_SOLUTION
# First we define the middle square class.
class RNGMiddleSquare(RNG):
    # For our state, we need the previously generated integer and what
    # `n` we are using.
    def __init__(self, seed, n):
        super().__init__(seed)
        self.state = seed
        self.n = 2

    # Here, we define the actual RNG.
    def __call__(self):
        # The `zfill` method pads the string with 0s, this gives us a string.
        self.state = str(self.state**2).zfill(2 * self.n)
        # Next, we extract the middle of the string as an integer.
        i = int(self.n / 2)
        self.state = int(self.state[i:-i])
        # We divide by 10^n - 1 so our RN is between 0 and 1.
        return self.state / float(10**self.n - 1)

In [ ]:
# Now we can try it out for n = 2.
# Input the random number seed and define the sequence.
seed = int(input("Please enter a two-digit number:\n[##] "))

# Create the RNG.
rng = RNGMiddleSquare(seed, 2)

# We store the current RN in `rn` and the sequence of RNs in `rns`.
rn, rns = None, []

# Start throwing random numbers.
while rn not in rns:
    # Append RN to the sequence.
    if rn != None:
        rns.append(rn)
    # Generate the new random number using the middle square method.
    rn = rng()
    # Print the result.
    print(f"#{len(rns)}: {rn}")

# Print the result.
print(
    f"We began with {seed} and"
    f" have repeated ourselves after {len(rns)} steps"
    f" with {rn}."
)

# We can now calculate the mean and the variance of the sequence.
# Numpy allows us to easily calculate the mean and variance.
print("mean = ", mean(rns))
print("variance = ", variance(rns))

In [ ]:
# We can also try it for n = 4.
# Input the random number seed and define the sequence.
seed = int(input("Please enter a four-digit number:\n[####] "))

# Create the RNG.
rng = RNGMiddleSquare(seed, 4)

# We store the current RN in `rn` and the sequence of RNs in `rns`.
rn, rns = None, []

# Start throwing random numbers.
while rn not in rns:
    # Append RN to the sequence.
    if rn != None:
        rns.append(rn)
    # Generate the new random number using the middle square method.
    rn = rng()
    # Print the result.
    print(f"#{len(rns)}: {rn}")

# Print the result.
print(
    f"We began with {seed} and"
    f" have repeated ourselves after {len(rns)} steps"
    f" with {rn}."
)

In [ ]:
# Now, we try out all the seeds for n = 2.
# Create a dictionary of periods.
periods = {}

# For all seeds we iterate between 1 and 99.
# We don't use 0, since this has a period of 0.
for seed in range(1, 100):
    # Create the RNG.
    rng = RNGMiddleSquare(seed, 2)

    # Determine the period.
    rns = rng_sequence(rng)

    # Include the period and sequence in the dictionary.
    period = len(rns)
    if not period in periods:
        periods[period] = [[seed, rns]]
    else:
        periods[period] += [[seed, rns]]

# Print out a sorted list of the periods.
for period, seeds in sorted([(key, val) for key, val in periods.items()]):
    print("-" * 10)
    print(f"period = {period}")
    print("-" * 10)
    for seed, rns in seeds:
        print(f"seed = {seed}")
        if len(rns) > 0:
            print("  mean = ", mean(rns))
            print("  variance = ", variance(rns))
###STOP_SOLUTION

## RNGs based on modular arithmetic

The middle square can have short periods (bad) and are not even very random (really bad).

These types of RNGs were quickly overshadowed by those based on modular arithmetic. Two numbers are of the same modular class if they differ by only multiples of an integer $m$. Thus,

$$1 \equiv 4\pmod{3} \equiv 7\pmod{3} \equiv 82\pmod{3}.$$

In Python, the mod operator is given by the `%` symbol. Modular arithmetic has some nice properties that allow us to calculate or estimate periods, and it can be done quickly on a computer (a feature that could be relevant when sampling millions of RNs). However, the equivalences can also lead to some unexpected consequences.

First, we will explore multiplicative congruential generators.
These have the form,

$$x_i = a x_{i-1} \mod m, 0 < x_i < m \in \mathbb{Z}.$$

As with all such algorithms, you can convert this into a float between $0$ and $1$ by dividing by the largest element $m$. $a$ and $m$ are integers, with $a$ relatively prime to $m$.

In [ ]:
# We can now define a multiplicative congruential generator.
class RNGMultCon(RNG):
    # For our state, we only need the previously generated integer.
    def __init__(self, seed, a, m):
        super().__init__(seed)
        self.state = seed
        self.a = a
        self.m = m

    # Here, we define the actual RNG.
    def __call__(self):
        self.state = (self.a * self.state) % self.m
        return self.state / float(self.m - 1)

Now, let us test this RNG by first checking its period, mean, and variance.

In [ ]:
# Create the RNG.
rng = RNGMultCon(9, 3, 31)

# Find the period.
rns = rng_sequence(rng)

# Print the period, mean, and variance.
print(f"period = {len(rns)}")
print(f"mean = {mean(rns)}")
print(f"variance = {variance(rns)}")

### Exercise: patterns in multiplicative congruential generator

Lets investigate some other properties of this RNG.
The pairs $(x_i,x_{i-1})$ lie on a plane. Plot their pattern.

In [ ]:
###START_EXERCISE
# We can use `matplotlib` to create a figure and axis.
fig, ax = plt.subplots()

# Use the `scatter` method of `ax` to plot the RNs.
xs = [0]
ys = [0]
ax.scatter(xs, ys)
###STOP_EXERCISE

In [ ]:
###START_SOLUTION
# Our x values are just the random numbers.
xs = rns

# Our y values are shifted to the left by one.
ys = rns[-1:] + rns[:-1]
# This can also be accopmlished with `numpy.roll`.
# ys = numpy.roll(rns, 1)

# Create the plot.
fig, ax = plt.subplots()
ax.scatter(xs, ys)

# Label each RN by its number in the sequence.
for i, rn in enumerate(rns):
    ax.annotate(str(i), (xs[i], ys[i]))
###STOP_SOLUTION

### Exercise: choosing better parameters

A better choice of $a$ and $m$ can give our RNG better properties.
Create a new RNG with $a = 65539$ and $m = 2^{31}$. Generate a sequence of 1000 RNs using seed = $2^8 - 1$. Make a sequence of points $(x_i, x_{i-1})$ and plot them. Do any patterns emerge?

In [ ]:
###START_EXERCISE
# You already have the RNGMultCon class, so use this, and then plot.
###STOP_EXERCISE

In [ ]:
###START_SOLUTION
# Create the random number generator.
rng = RNGMultCon(2**8 - 1, 65539, 2**31)

# Generate the sequence.
rns = rng_sequence(rng, 1000)

In [ ]:
# Create the pairs.
xs = rns
ys = rns[-1:] + rns[:-1]

# Create the plot.
fig, ax = plt.subplots()
ax.scatter(xs, ys)
###STOP_SOLUTION

Now, given a little more advanced knowledge, let us see if we can find a pattern. Using the same RNG and seed, generate 20002 RNs and make a sequence of points $(x_{i-1}, x_i, x_{i+1})$. Filter these points for $0.50 < x_i < 0.51$. Make a 2-D scatter plot of $(x_{i-1},x_{i+1})$ for these filtered points. Do any patterns emerge?

In [ ]:
###START_SOLUTION
# Create the random number generator.
rng = RNGMultCon(2**8 - 1, 65539, 2**31)

# Generate the sequence.
rns = rng_sequence(rng, 20002)

In [ ]:
# Create the pairs.
xs = rns[-1:] + rns[:-1]
ys = rns[1:] + rns[:1]

# Filter the pairs.
xs = [x for x, rn in zip(xs, rns) if 0.50 < rn < 0.51]
ys = [y for y, rn in zip(ys, rns) if 0.50 < rn < 0.51]

# Create the plot.
fig, ax = plt.subplots()
ax.scatter(xs, ys)

# Indeed, we do see a pattern here!
###STOP_SOLUTION

So far, we have only been looking in two dimensions, what about in three? Generate 1000 RNs using the same RNG and seed. Then make a 3-D scatter plot with the points $(x_{i-1},x_i,x_{i+1})$.

In [ ]:
###START_EXERCISE
# The following will create a 3-D scatter plot.

# Set your points.
xs = [0]
ys = [0]
zs = [0]

# Create the figure and allow to be 3-D.
fig = plt.figure()
ax = fig.add_subplot(projection="3d")

# Create the scatter plot.
ax.scatter3D(xs, ys, zs)
###STOP_EXERCISE

In [ ]:
###START_SOLUTION
# Create the random number generator.
rng = RNGMultCon(2**8 - 1, 65539, 2**31)

# Generate the sequence.
rns = rng_sequence(rng, 1000)

In [ ]:
# Create the triplets.
xs = rns[-1:] + rns[:-1]
ys = rns
zs = rns[1:] + rns[:1]

# Create the figure.
fig = plt.figure()
ax = fig.add_subplot(projection="3d")
ax.scatter3D(xs, ys, zs)

# Rotate the view so we can see the pattern.
ax.view_init(elev=7, azim=-124)
###STOP_SOLUTION

This example is not just pedagogical. In fact, this RNG is called `RANDU`. The authors of Numerical Recipes (3rd ed), share this anecdote:

> Even worse, many early generators happened to make particularly bad choices for m and a. One infamous such routine, `RANDU`, with a = 65539 and m = $2^{31}$, was widespread on IBM mainframe computers for many years, and widely copied onto other systems. One of us recalls as a graduate student producing a "random" plot with only 11 planes and being told by his computer center's programming consultant that he had misused the random number generator: "We guarantee that each number is random individually, but we don't guarantee that more than one of them is random." That set back our graduate education by at least a year!

## Combined Generators

`Individual` RNGs can have correlations.
A solution to this is to scramble the results again with a second RNG.
In fact, you could make the algorithm very complicated by using multiplication or some other math functions. The cost is the ease of programming the algorithm AND, more importantly,
the execution time. In many applications, the RNG is the bottleneck.

... More details here about `Fibonacci` RNGs

Also, some recommendations on how to combine.

## XORshift RNGs

Modern RNGs still combine two algorithms to remove undesired correlations.
However, they use an independent algorithm for the first sequence, so as not to unwittingly combine two correlations that arise from the same `type` of algorithm.

One such popular algorithm is called `XORshift`. Its properties are understood by studying the multiplication of 3 special kinds of 32- or 64-dimensional binary matrices, but it can be programmed easily using bit shift and XOR operations. The resulting algorithm does not look anything like matrix multiplication, but it really is. This is because a bit shift can be represented on an n-bit vector by a matrix with only ones on a sub-diagonal. Thus, to right-shift a bit sequence $\beta = (b_1,b_2,\cdots,b_n) \to \beta^{'} = (0,b_1,b_2,\cdots,b_{n-1})$, you would right multiply by an $n\times n$ matrix with only $1$s above the diagonal:
$$
\begin{pmatrix}
0 & 1 & 0 & \cdots &  0 \\
0 & 0 & 1 & \cdots &  0 \\
0 & 0 & 0 & 1      &  0 \\
\vdots & \vdots & \vdots & \ddots &  1 \\
0 & 0 & 0 & 0      &                 0  \\
\end{pmatrix}
$$

Similarly, a left-shift matrix has only $1$s on a subdiagonal `below` the diagonal.
Finally, since these are binary matrices, all operations use  integer arithmetic modulo 2.

The claim is that the series of operations $\beta T, \beta T^2 , \cdots \beta T^{2^n-1}$, every possible $\beta$ is produced.
This means that the cycle has length $2^n-1$.

For the current implementations, $T = (1_n+L^a)(1_n+R^b)(1_n+L^c)$, an $n\times n$ binary matrix with $(a,b,c)$ bit-shifts left-right-left.
Only certain triplets $(a,b,c)$ have the desired property.

A minimal test that $T$ has the desired properties is to square $T$ $n$ times and check if it is equal to $T$.

Perform this test for the tuples $(1,3,10)$, $(5,17,13)$, and $(2,5,14)$ using 32-bit precision.
Which of these are suitable triplets?

In [ ]:
import numpy as np
import scipy as sp

In [ ]:
def make_matrix(a, b, c):
    from scipy.sparse import diags

    E = sp.sparse.eye(32, dtype=np.int32)
    T1 = diags([1], [a], shape=(32, 32), dtype=np.int32) + E
    T2 = diags([1], [-b], shape=(32, 32), dtype=np.int32) + E
    T3 = diags([1], [c], shape=(32, 32), dtype=np.int32) + E
    #
    return T1 @ T2 @ T3

In [ ]:
T = make_matrix(5, 17, 13)
U = T.todense()
for i in range(0, 32):
    U = U @ U % 2
print((T - U).nonzero())

Use (a,b,c) = (5,17,13)

Choose a number < 10**32-1 and represent it as an `np.array` of length 32, i.e. `3 = [0,0,....0,1,1]`.

Right multiply (mod 2) by T and convert the result back to an integer.

Now, starting with the integer i, perform the operations:
i = i ^ i>>a i = i ^ i<<b i = i ^ i<<c

Show the results are equivalent.
Hint: make sure your integer doesn't become int64!

In [ ]:
def func(x, bits):
    return np.array([int(i) for i in bin(x)[2:].zfill(bits)])

In [ ]:
numb = np.array(2**30 - 3, dtype=np.int32)

In [ ]:
print(func(numb, 32))

In [ ]:
print(c), bin(c)
print(func(c, 32))

In [ ]:
c = numb ^ numb >> 5
c = c.astype(np.int32)
c = c ^ (c << 17 & 0xFFFFFFFF)
c = c.astype(np.int32)
c = c ^ c >> 13
c = c.astype(np.int32)

In [ ]:
b = func(numb, 32)

In [ ]:
f = (b @ T) % 2

In [ ]:
print(f)

Implement your own high quality RNG using the triplet (17,31,8).
Combine it with another RNG to make it safer.

In [ ]:
# Need help or must use c-functions
Ran:
    def __init__(self, seed):
        self.v = np.array(4101842887655102017, dtype=np.ulonglong)
        self.u = np.array(np.ulonglong(seed) ^ self.v, dtype=np.ulonglong)
        self.w = np.array(1, dtype=np.ulonglong)
        self.int64()
        self.v = self.u
        self.int64()
        self.w = self.v
        self.int64()

    def int64(self):
        self.u = self.u * 2862933555777941757 + np.ulonglong(7046029254386353087)
        self.v ^= self.v >> 17
        self.v ^= self.v << np.uint64(31)
        self.v ^= self.v >> np.uint64(8)
        self.w = np.uint32(4294957665) * (np.uint32(self.w) & 0xFFFFFFFF) + (
            np.uint32(self.w) >> 32
        )
        x = np.uint64(self.u) ^ np.uint64(np.uint64(self.u) << np.uint64(21))
        x ^= x >> np.uint64(35)
        x ^= x << np.uint64(4)
        state = np.uint64(x + self.v) ^ np.uint64(self.w)
        return state

    def doub(self):
        return 5.42101086242752217e-20 * self.int64()

    def int32(self):
        return np.uint32(self.int64())

In [ ]:
myran = Ran(17)

## Tests of Random Number Generators

Diehard, NIST references.
Craps test?